In [1]:
import os
import logging
import fitz
from fastapi import FastAPI, UploadFile, HTTPException, File, Form ,UploadFile, Depends
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from dotenv import load_dotenv
from typing import List
import openai
import uvicorn
import tempfile
import time
from pymongo import MongoClient , DESCENDING
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from datetime import datetime
from rank_bm25 import BM25Okapi
from langchain_openai import OpenAI
import re
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import CharacterTextSplitter
from enum import Enum
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFium2Loader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
import pickle
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import ChatOpenAI


d:\RAG\Fast-RAG\app\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
import json
def load_mappings(json_file_path):
    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        return {entry['clear_text']: entry['uuid'] for entry in data}
    except FileNotFoundError:
        print(f"Error: The file {json_file_path} was not found.")
        return {}
    except json.JSONDecodeError:
        print(f"Error: The file {json_file_path} is not a valid JSON file.")
        return {}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

In [7]:


def load_mappings(json_file_path):
    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        return {entry['clear_text']: entry['uuid'] for entry in data}
    except FileNotFoundError:
        print(f"Error: The file {json_file_path} was not found.")
        return {}
    except json.JSONDecodeError:
        print(f"Error: The file {json_file_path} is not a valid JSON file.")
        return {}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}

# Function to replace text in documents
def replace_text_in_documents(docs, mappings):
    for doc in docs:
        content = doc.page_content
        for clear_text, uuid in mappings.items():
            content = content.replace(clear_text, uuid)
        doc.page_content = content
    return docs





In [10]:
json_file_path = 'mask.json'
mappings = load_mappings(json_file_path)

In [11]:
text_splitter = SemanticChunker(OpenAIEmbeddings())

loader=PyMuPDFLoader('RAG Project.pdf')
docs=loader.load_and_split(text_splitter=text_splitter)
mappings = load_mappings(json_file_path)

In [13]:
docs = replace_text_in_documents(docs, mappings)

In [14]:
for doc in docs:
    print(doc.page_content)

Develop 4d026f91-061d-48f3-ad88-1dd1e3dcc87a (API) which can ingest PDF, Text files into multiple collections with date 
and time. It will be hosted on a normal computer with 4 core and 18 GB ram (not 
GPU). Multiple collection means, we can have separate collection for different 
purpose so there is sufficient privacy of data. The ingest should be effective to use best algorithms available open source to keep 
ingest cost to minimum. PDF files can be large, few hundred pages, need effective solution.
Provide ingest parameters to support efficiency like chunk_size, chunk_overlap, 
chunk_method, embed_method. RAPTOR
Smentic Clustering
Hybrrid 4d026f91-061d-48f3-ad88-1dd1e3dcc87a
Pre processing 
RERANK based on timestamp
Contexual compression
Create an ingest method which takes mask.json , it will have the information in 
following format 
   {
        "data_type": "name",
        "clear_text": "John Doe",
        "uuid": "4d026f91-061d-48f3-ad88-1dd1e3dcc87a"
    },
    {
        "data_

In [2]:
vectorstore=PineconeVectorStore(index_name="dui",embedding =OpenAIEmbeddings())

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'x-pinecone-api-version': '2024-04', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': 'e3dce8bbd201b3a6b23854dbbb1ec4d7', 'Date': 'Fri, 05 Jul 2024 07:58:20 GMT', 'Server': 'Google Frontend', 'Content-Length': '78', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource dui not found"},"status":404}


In [ ]:
llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=vectorstore.as_retriever()
)

compressed_docs = compression_retriever.invoke(
    "What can you tell me about the various parameters of Chatbot"
)

In [ ]:
compressed_docs

[Document(page_content='●\n/chat_gpt\no\nprompt\no\ntemperature\no\nstructure (json format)\no\nhistory\n●\n/local_llm\no\nprompt\no\ntemperature\no\nstructure (json format)\no\nhistory', metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '/tmp/tmpy6dfahoo', 'format': 'PDF 1.4', 'keywords': '', 'modDate': '', 'page': 2.0, 'producer': 'Skia/PDF m128 Google Docs Renderer', 'source': '/tmp/tmpy6dfahoo', 'subject': '', 'title': 'RAG Project', 'total_pages': 3.0, 'trapped': ''}),
 Document(page_content='- Create few endpoints for maintenance of RAG system like\nEndpoint list\n●\n/list_all_collections\n●\n/delete_collection\n●\n/delete_all_collections\nCreate RAG query endpoint to have conversation with ingest data and obtain output\nin very defined json format.\nEndpoint list\n●\n/query_chat_gpt\no\ncollection\no\nprompt\no\nn_results\no\ntemperature\no\nstructure (json format)\no\nhistory\no\ndate\no\ntime\n●\n/query_local_llm\no\ncollection\no\nprompt\no\nn_results\no

In [ ]:
question="What can you tell me about the various parameters of Chatbot"

In [ ]:
from langchain.retrievers.multi_query import MultiQueryRetriever
llm = ChatOpenAI(model='gpt-4o',temperature=0)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(), llm=llm
)


In [ ]:
retriever_from_llm.invoke(question)

[Document(page_content='●\n/chat_gpt\no\nprompt\no\ntemperature\no\nstructure (json format)\no\nhistory\n●\n/local_llm\no\nprompt\no\ntemperature\no\nstructure (json format)\no\nhistory\nProvide detailed installation documentation to deploy endpoints on GPU and on local\ncloud server.', metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': '/tmp/tmpy6dfahoo', 'format': 'PDF 1.4', 'keywords': '', 'modDate': '', 'page': 2.0, 'producer': 'Skia/PDF m128 Google Docs Renderer', 'source': '/tmp/tmpy6dfahoo', 'subject': '', 'title': 'RAG Project', 'total_pages': 3.0, 'trapped': ''}),
 Document(page_content='The retrieval should be effective to make queries to get most efficient data from\nvector store on similarity with simple query, may be use local tiny LLM to have\nintelligent query or some sort of recursive approach to have intelligent data.\nCreate few endpoints for maintenance of RAG system like\nEndpoint list\n●\n/list_all_collections\n●\n/delete_collection\n●\n/delete

In [ ]:
from typing import List

from langchain.chains import LLMChain
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field


# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)


output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)
llm = ChatOpenAI(temperature=0)

# Chain
llm_chain = LLMChain(llm=llm, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "What are the approaches to Task Decomposition?"



/home/waleed_butt/abark/Fast-RAG/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

llm([HumanMessage(content="""You are an AI language model assistant. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines. Seprate each question by a new line.
    Original question: """+question)])

AIMessage(content='What are the different parameters involved in configuring a Chatbot?\nCan you explain the various settings and parameters used in Chatbot development?\nWhat are the key parameters to consider when designing a Chatbot?\nCould you provide details on the different configuration options for Chatbots?\nWhat are the essential parameters that influence the performance of a Chatbot?', response_metadata={'token_usage': {'completion_tokens': 66, 'prompt_tokens': 98, 'total_tokens': 164}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_ce0793330f', 'finish_reason': 'stop', 'logprobs': None}, id='run-701d2a77-0067-4681-9991-52bbfcedaadb-0', usage_metadata={'input_tokens': 98, 'output_tokens': 66, 'total_tokens': 164})

In [ ]:
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.invoke(input="What does the course say about regression?")
len(unique_docs)